In [10]:
import numpy as np
import os
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio
import zipfile
import shutil
import time
import contextlib

## Crop images

In [14]:
def crop_images(in_filenames, out_filenames, patch_size = 512, m = 0, overlap = 0.5):

    im = gdal.Open(in_filenames[0])
    gt = gdal.Open(in_filenames[1])
    
    k = m 
    magnitude = 5
    l = 0
    for i in np.arange(0, im.RasterXSize, int(patch_size*(1-overlap))):
        for j in np.arange(0, im.RasterYSize,int(patch_size*(1-overlap))):
            
            # Hardcode so the images will fit together
            j_gt = im.RasterYSize - (j + patch_size)
            # j_gt = j
            
            s = '{:0'+str(magnitude)+'d}'
            gdal.Translate(out_filenames[1] + s.format(k)+'.tif', gt, srcWin = [i, j_gt, patch_size, patch_size])
            l += 1
            
            with rasterio.open(out_filenames[1] + s.format(k)+'.tif') as src:
                cashew = src.read()
                ten_perc = cashew.sum() > (patch_size**2)*0.1
                src.close()
            if ten_perc:
                gdal.Translate(out_filenames[0] + s.format(k)+'.tif', im, srcWin = [i, j, patch_size, patch_size])
                k+=1
            else:
                with contextlib.suppress(PermissionError):
                    os.remove(out_filenames[1] + s.format(k)+'.tif')

    im = None
    gt = None
            
    return k
                

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

## For Tanzania

In [15]:
if 'Tanzania' not in os.listdir('../Data/Planet/'):
    os.mkdir('../Data/Planet/Tanzania')

In [17]:
# Crop and save planet and GT images for Tanzania
dir = '../Data/Planet/'
sets = ['Train', 'Validation', 'Test']

for s in sets:
    
    m = 0
    
    for i in os.listdir(dir):
        
        if 'Tanzania' + s in i:

            planet_path = dir + i
            vect_path = '../Data/Vector/Tanzania'+s+'Rasterized'+i.split('.')[0][-1]+'.tif'
            
            k = crop_images([planet_path, vect_path], 
                            ['../Data/Planet/Tanzania/CroppedTanzania'+s+'StudyArea_', '../Data/Planet/Tanzania/CroppedTanzania'+s+'GT_'], patch_size=256, m = m, overlap = 0.75)
            m = k

            # with contextlib.suppress(PermissionError):
            #     os.remove(planet_path)
            #     os.remove(vect_path)

    print(s, m)

Train 997
Validation 448
Test 258


In [18]:
# Zip folder with planet and GT images
with zipfile.ZipFile('../Data/PlanetTanzania.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('../Data/Planet/Tanzania', zipf)

In [ ]:
with contextlib.suppress(PermissionError):
        os.remove('../Data/Planet/PlanetTilesIvory_' + str(i+1) + '.tif')
        os.remove('../Data/Vector/IvoryCoastRasterized_' + str(i+1) + '.tif')

## For Ivory Coast

In [6]:
if 'IvoryCoast' not in os.listdir('../Data/Planet/'):
    os.mkdir('../Data/Planet/IvoryCoast')

In [7]:
m  = 0
for i in range(4):
    k = crop_images(in_filenames = ['../Data/Planet/PlanetTilesIvory_' + str(i+1) + '.tif', '../Data/Vector/IvoryCoastRasterized_' + str(i+1) + '.tif'], 
                    out_filenames = ['../Data/Planet/IvoryCoast/CroppedIvoryCoastStudyArea_', '../Data/Planet/IvoryCoast/CroppedIvoryCoastGT_'],
                    patch_size = 256, m = m)
    m = k
    print(m)
    with contextlib.suppress(PermissionError):
        os.remove('../Data/Planet/PlanetTilesIvory_' + str(i+1) + '.tif')
        os.remove('../Data/Vector/IvoryCoastRasterized_' + str(i+1) + '.tif')

230
833
1121
1629


In [8]:
with zipfile.ZipFile('../Data/PlanetIvoryCoast.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('../Data/Planet/IvoryCoast', zipf)

In [12]:
shutil.rmtree('../Data/Planet/IvoryCoast')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../Data/Planet/IvoryCoast'